Projeto em PySpark para dados de preços de combustíveis da ANP.

O trabalho faz uso de dados publicados pela ANP (Agência Nacional de Petróleo)

> Em cumprimento às determinações da Lei do Petróleo (Lei nº 9478/1997, artigo 8º), a ANP acompanha os preços praticados por revendedores de combustíveis automotivos e de gás liquefeito de petróleo envasilhado em botijões de 13 quilos (GLP P13), por meio de uma pesquisa semanal de preços realizada por empresa contratada.

- [Série Histórica de Preços de Combustíveis e de GLP](https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis)

Coletamos a série histórica de "Combustíveis automotivos" que vai de 2004 a 2023. São 39 arquivos CSV totalizando aproximadamente 3.7 GB.

![Amostra dos dados](./assets/amostra_planilha_revenda.png)


Aproveitando a configuração de cluster da Google Cloud Platform criada para o [projeto Hadoop](https://github.com/pinei/hadoop-precos-combustiveis), incluímos o JupyterLab para trabalhar com o PySpark.

![cluster-info](./assets/cluster_info.png)